# Data Pre-processing and Graph/Chart Generation

## Importing libraries and the Tweet dataset

In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

df = pd.read_excel("dataset.xlsx").copy()

### Data pre-processing for the Tweet dataset

In [36]:
# Check which columns have null values
# print(df.isna().any()) # Should only be True for 'Account bio' and 'Location'

# Dealing with null values
df["Account bio"] = df["Account bio"].fillna("No bio")
df["Location"] = df["Location"].fillna("No location")

# Joined
for i in range(len(df)):
    if isinstance(df["Joined"][i], str):
        date = df["Joined"][i].split("/")
        df["Joined"][i] = f"{('0' * (2 - len(date[0]))) + date[0]}/{df['Joined'][i][-2:]}"
    else:
        date = str(df["Joined"][i]).split()[0].split("-")
        df["Joined"][i] = f"{date[1]}/{date[2]}"

# Interactions
df["Interactions"] = df["Likes"] + df["Replies"] + df["Retweets"]

# Date posted
for i in range(len(df)):
    if isinstance(df["Date posted"][i], str):
        date = df["Date posted"][i].split()[0].split("/")
        df["Date posted"][i] = int(date[1]) + (12 * (int(date[2][2:] if len(date[2]) > 2 else date[2]) - 16))
        # df["Date posted"][i] = f"{('0' * (2 - len(date[1]))) + date[1]}/{date[2][2:] if len(date[2]) > 2 else date[2]}"
    else:
        date = str(df["Date posted"][i]).split()[0].split("-")
        df["Date posted"][i] = int(date[2]) + (12 * (int(date[0][2:]) - 16))
        # df["Date posted"][i] = f"{date[2]}/{date[0][2:]}"

C:\Users\josh\AppData\Local\Temp\ipykernel_9396\1351837390.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\josh\AppData\Local\Temp\ipykernel_9396\1351837390.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\josh\AppData\Local\Temp\ipykernel_9396\1351837390.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\josh\AppData\Local\Temp\ipykernel_9396\1351837390.py:28: SettingWithCopyWarning:


A value is 

### Creating a new dataframe based on the Tweet dataset binned by months

In [37]:
mm_yy_date = [f"{i if i >= 10 else '0' + str(i)}/{j}" for j in range(16, 23) for i in range(1, 13)]
mm_yy = [i for i in range(1, 85)]
date_df = pd.DataFrame(mm_yy, columns=["Month"])
date_df["Month_date"] = mm_yy_date
date_df["Count"] = date_df["Month"].map(df["Date posted"].value_counts())
date_df["Count"] = date_df["Count"].fillna(0)
date_df["Interactions"] = date_df["Month"].map(df.groupby("Date posted")["Interactions"].sum())
date_df["Interactions"] = date_df["Interactions"].fillna(0)

temp = list(date_df["Count"])
m = [2, 3, 4, 5, 6, 38, 39, 40, 41, 42, 74, 75, 76, 77, 78]
date_df["ep"] = np.asarray([temp[i] if i in m else None for i in range(84)])

### Generating a heatmap for the whole Tweet dataset and another heatmap focusing on Followers and Interactions

In [38]:
fig = px.imshow(df.corr(), text_auto=True, title="df dataset numerical variable correlation")
fig.show()
# fig.write_html("dfcorr.html")
fig = px.imshow(df[["Followers", "Interactions"]].corr(), text_auto=True, title="Tweet dataset correlation between Account Followers and Tweet Interactions")
fig.show()
# fig.write_html("dfcorr_followers_interactions.html")

### Generating a histogram for Account type, various violin plots for numerical variables, and a bar graph detailing total interactions per tweet

In [39]:
fig = px.histogram(df, x="Account type", title="'Account type' Histogram")
fig.show()
# fig.write_html("hist_acctype.html")
fig = px.violin(df, y="Followers", title="'Followers' Violin Plot")
fig.show()
# fig.write_html("violin_followers.html")
fig = px.violin(df, y="Following", title="'Following' Violin Plot")
fig.show()
# fig.write_html("violin_following.html")
fig = px.violin(df, y="Likes", title="'Likes' Violin Plot")
fig.show()
# fig.write_html("violin_likes.html")
fig = px.violin(df, y="Retweets", title="'Retweets' Violin Plot")
fig.show()
# fig.write_html("violin_retweets.html")
fig = px.violin(df, y="Replies", title="'Replies' Violin Plot")
fig.show()
# fig.write_html("violin_replies.html")
fig = px.bar(df, y=["Likes", "Retweets", "Replies"], title="'Interactions' Bar Graph detailing 'Likes', 'Retweets', and 'Replies'")
fig.show()
# fig.write_html("bar_interactions.html")

### Generating a heatmap for the binned dataframe, a line graph showing tweet frequency per month, and a scatter plot for tweet interactions per month

In [45]:

fig = px.imshow(date_df.corr(), text_auto=True, title="Binned Monthly Tweet dataset numerical variable correlation")
fig.show()
# fig.write_html("datedfcorr.html")
fig = go.Figure(data=go.Scatter(x=date_df["Month_date"], y=date_df["Count"],mode='lines', line_color='#ff6f00', name="Non-election Period"))
fig.add_trace(go.Scatter(x=date_df["Month_date"], y=date_df["ep"],mode='lines', line=dict(color="#0000ff"), name="Election Period"))
fig.update_layout(title="Number of Tweets per Month", xaxis_title="Months", yaxis_title="Tweet Count")
fig.show()
# fig.write_html("tweetfrequency.html")
fig = px.scatter(date_df, x="Month", y="Interactions", color="Count", title="Scatter plt showing total tweet interactions per month")
fig.show()
# fig.write_html("scatterinteraction.html")